In [1]:
# Object Detecion 
import cv2
from ultralytics import YOLO
#plots
import matplotlib.pyplot as plt
import seaborn as sns

#basics
import pandas as pd
import numpy as np
import os
import subprocess

from tqdm.notebook import tqdm

# Display image and videos
import IPython
from IPython.display import Video, display
%matplotlib inline

In [2]:
#loading a YOLO model 
model = YOLO('/home/raza.imam/Documents/FRC/YOLOv8/runs/detect/train/weights/best.pt')

#geting names from classes
dict_classes = model.model.names

In [6]:
dict_classes

{0: 'camel', 1: 'mask', 2: 'pole', 3: 'rope'}

In [4]:
class_IDS = [0]
frame_path = '/home/raza.imam/Documents/FRC/YOLOv8/datasets/Camel-object-detection-MAIN-2/test/images/Camel_6_jpg.rf.3642acb12e8961266c3b0318e6b275b4.jpg'
frame = cv2.imread(frame_path)

In [9]:
veiculos_contador_in = dict.fromkeys(class_IDS, 0)
veiculos_contador_out = dict.fromkeys(class_IDS, 0)
frames_list = []

y_hat = model.predict(frame, conf = 0.25, classes = class_IDS, device = 0, verbose = False)
    
    # Getting the bounding boxes, confidence and classes of the recognize objects in the current frame.
boxes   = y_hat[0].boxes.xyxy.cpu().numpy()
conf    = y_hat[0].boxes.conf.cpu().numpy()
classes = y_hat[0].boxes.cls.cpu().numpy()

positions_frame = pd.DataFrame(y_hat[0].cpu().numpy().boxes.boxes, columns = ['xmin', 'ymin', 'xmax', 'ymax', 'conf', 'class'])
    
#Translating the numeric class labels to text
labels = [dict_classes[i] for i in classes]

for ix, row in enumerate(positions_frame.iterrows()):
        # Getting the coordinates of each vehicle (row)
        xmin, ymin, xmax, ymax, confidence, category,  = row[1].astype('int')
        
        # Calculating the center of the bounding-box
        center_x, center_y = int(((xmax+xmin))/2), int((ymax+ ymin)/2)
        
        # drawing center and bounding-box of vehicle in the given frame 
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255,0,0), 5) # box
        cv2.circle(frame, (center_x,center_y), 5,(255,0,0),-1) # center of box
        
        #Drawing above the bounding-box the name of class recognized.
        cv2.putText(img=frame, text=labels[ix]+' - '+str(np.round(conf[ix],2)),
                    org= (xmin,ymin-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 0),thickness=2)
        
        
#updating the counting type of vehicle 
contador_in_plt = [f'{dict_classes[k]}: {i}' for k, i in veiculos_contador_in.items()]
contador_out_plt = [f'{dict_classes[k]}: {i}' for k, i in veiculos_contador_out.items()]

#drawing the number of vehicles in\out
cv2.putText(img=frame, text='N. vehicles In', 
            org= (30,30), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
            fontScale=1, color=(255, 255, 0),thickness=1)

cv2.putText(img=frame, text='N. vehicles Out', 
            org= (int(2800 * 5/100 ),30), 
            fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=1)
frames_list.append(frame)
        
#printing the frame
cv2.imshow('frame', frame)
cv2.waitKey(0)
#cv2.destroyAllWindows()

error: OpenCV(4.7.0) /io/opencv/modules/highgui/src/window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [5]:
# Auxiliary functions
def risize_frame(frame, scale_percent):
    """Function to resize an image in a percent scale"""
    width = int(frame.shape[1] * scale_percent / 100)
    height = int(frame.shape[0] * scale_percent / 100)
    dim = (width, height)

    # resize image
    resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    return resized

In [7]:
### Configurations
#Verbose during prediction
verbose = False
# Scaling percentage of original frame
scale_percent = 100


#-------------------------------------------------------
# Reading video with cv2
video = cv2.VideoCapture('/home/muhammad.huzaifa/Documents/Research/Internship/data/H.mp4')

# Objects to detect Yolo
class_IDS = [0] 
# Auxiliary variables
centers_old = {}
centers_new = {}
obj_id = 0 
veiculos_contador_in = dict.fromkeys(class_IDS, 0)
veiculos_contador_out = dict.fromkeys(class_IDS, 0)
end = []
frames_list = []
cy_linha = int(1500 * scale_percent/100 )
cx_sentido = int(2000 * scale_percent/100) 
offset = int(8 * scale_percent/100 )
contador_in = 0
contador_out = 0
print(f'[INFO] - Verbose during Prediction: {verbose}')


# Original informations of video
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)
print('[INFO] - Original Dim: ', (width, height))

# Scaling Video for better performance 
if scale_percent != 100:
    print('[INFO] - Scaling change may cause errors in pixels lines ')
    width = int(width * scale_percent / 100)
    height = int(height * scale_percent / 100)
    print('[INFO] - Dim Scaled: ', (width, height))
    

#-------------------------------------------------------
### Video output ####
video_name = 'result.mp4'
output_path = "rep_" + video_name
tmp_output_path = "tmp_" + output_path
VIDEO_CODEC = "MP4V"

output_video = cv2.VideoWriter(tmp_output_path, 
                               cv2.VideoWriter_fourcc(*VIDEO_CODEC), 
                               fps, (width, height))


#-------------------------------------------------------
# Executing Recognition 
for i in tqdm(range(int(video.get(cv2.CAP_PROP_FRAME_COUNT)))):
    
    # reading frame from video
    _, frame = video.read()
    
    #Applying resizing of read frame
    frame  = risize_frame(frame, scale_percent)
    
    if verbose:
        print('Dimension Scaled(frame): ', (frame.shape[1], frame.shape[0]))

    # Getting predictions
    y_hat = model.predict(frame, conf = 0.25, classes = class_IDS, device = 0, verbose = False)
    
    # Getting the bounding boxes, confidence and classes of the recognize objects in the current frame.
    boxes   = y_hat[0].boxes.xyxy.cpu().numpy()
    conf    = y_hat[0].boxes.conf.cpu().numpy()
    classes = y_hat[0].boxes.cls.cpu().numpy() 
    
    # Storing the above information in a dataframe
    positions_frame = pd.DataFrame(y_hat[0].cpu().numpy().boxes.boxes, columns = ['xmin', 'ymin', 'xmax', 'ymax', 'conf', 'class'])
    
    #Translating the numeric class labels to text
    labels = [dict_classes[i] for i in classes]
    
    # Drawing transition line for in\out vehicles counting 
    cv2.line(frame, (0, cy_linha), (int(4500 * scale_percent/100 ), cy_linha), (255,255,0),8)
    
    # For each vehicles, draw the bounding-box and counting each one the pass thought the transition line (in\out)
    for ix, row in enumerate(positions_frame.iterrows()):
        # Getting the coordinates of each vehicle (row)
        xmin, ymin, xmax, ymax, confidence, category,  = row[1].astype('int')
        
        # Calculating the center of the bounding-box
        center_x, center_y = int(((xmax+xmin))/2), int((ymax+ ymin)/2)
        
        # drawing center and bounding-box of vehicle in the given frame 
        cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255,0,0), 5) # box
        cv2.circle(frame, (center_x,center_y), 5,(255,0,0),-1) # center of box
        
        #Drawing above the bounding-box the name of class recognized.
        cv2.putText(img=frame, text=labels[ix]+' - '+str(np.round(conf[ix],2)),
                    org= (xmin,ymin-10), fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 0, 0),thickness=2)
        
        # Checking if the center of recognized vehicle is in the area given by the transition line + offset and transition line - offset 
        if (center_y < (cy_linha + offset)) and (center_y > (cy_linha - offset)):
            if  (center_x >= 0) and (center_x <=cx_sentido):
                contador_in +=1
                veiculos_contador_in[category] += 1
            else:
                contador_out += 1
                veiculos_contador_out[category] += 1
    
    #updating the counting type of vehicle 
    contador_in_plt = [f'{dict_classes[k]}: {i}' for k, i in veiculos_contador_in.items()]
    contador_out_plt = [f'{dict_classes[k]}: {i}' for k, i in veiculos_contador_out.items()]
    
    #drawing the number of vehicles in\out
    cv2.putText(img=frame, text='N. vehicles In', 
                org= (30,30), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                fontScale=1, color=(255, 255, 0),thickness=1)
    
    cv2.putText(img=frame, text='N. vehicles Out', 
                org= (int(2800 * scale_percent/100 ),30), 
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=1)
    
    #drawing the counting of type of vehicles in the corners of frame 
    xt = 40
    for txt in range(len(contador_in_plt)):
        xt +=30
        cv2.putText(img=frame, text=contador_in_plt[txt], 
                    org= (30,xt), fontFace=cv2.FONT_HERSHEY_TRIPLEX, 
                    fontScale=1, color=(255, 255, 0),thickness=1)
        
        cv2.putText(img=frame, text=contador_out_plt[txt],
                    org= (int(2800 * scale_percent/100 ),xt), fontFace=cv2.FONT_HERSHEY_TRIPLEX,
                    fontScale=1, color=(255, 255, 0),thickness=1)
    
    #drawing the number of vehicles in\out
    cv2.putText(img=frame, text=f'In:{contador_in}', 
                org= (int(1820 * scale_percent/100 ),cy_linha+60),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=2)
    
    cv2.putText(img=frame, text=f'Out:{contador_out}', 
                org= (int(1800 * scale_percent/100 ),cy_linha-40),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(255, 255, 0),thickness=2)

    if verbose:
        print(contador_in, contador_out)
    #Saving frames in a list 
    frames_list.append(frame)
    #saving transformed frames in a output video formaat
    output_video.write(frame)
    
#Releasing the video    
output_video.release()


####  pos processing
# Fixing video output codec to run in the notebook\browser
if os.path.exists(output_path):
    os.remove(output_path)
    
subprocess.run(
    ["ffmpeg",  "-i", tmp_output_path,"-crf","18","-preset","veryfast","-hide_banner","-loglevel","error","-vcodec","libx264",output_path])
os.remove(tmp_output_path)

[INFO] - Verbose during Prediction: False
[INFO] - Original Dim:  (3840, 2160)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


  0%|          | 0/5348 [00:00<?, ?it/s]

[hevc @ 0x7f8e3fb239c0] Could not find ref with POC 0
[hevc @ 0x7f8d6a817f00] Could not find ref with POC 37
[hevc @ 0x7f8eecdcd740] Could not find ref with POC 0
[hevc @ 0x7f8e45c0e500] Could not find ref with POC 31
[hevc @ 0x7e17580] Could not find ref with POC 13
[hevc @ 0x7f8d8c9586c0] Could not find ref with POC 18
[hevc @ 0x803d2c0] Could not find ref with POC 32
[hevc @ 0x803d2c0] Could not find ref with POC 11
[hevc @ 0x7e17580] Could not find ref with POC 38
[hevc @ 0x7d7f940] Could not find ref with POC 19
[hevc @ 0x7e17580] Could not find ref with POC 29


In [ ]:
# Checking samples of processed frames
for i in [28, 29, 32, 40, 42, 50, 58]:
    plt.figure(figsize =( 14, 10))
    plt.imshow(frames_list[i])
    plt.show()

In [1]:
# Object Detecion 
import cv2
from ultralytics import YOLO
#plots
import matplotlib.pyplot as plt
import seaborn as sns

#basics
import pandas as pd
import numpy as np
import os
import subprocess

from tqdm.notebook import tqdm

# Display image and videos
import IPython
from IPython.display import Video, display
%matplotlib inline

In [11]:
import cv2

# Loading a YOLO model
model = YOLO('/home/muhammad.huzaifa/Documents/Research/Internship/runs/detect/train3/weights/best.pt')

# Getting names from classes
dict_classes = model.model.names

class_IDS = [0]
frame_path = 'data_img/frame1_2760.jpg'
frame = cv2.imread(frame_path)

y_hat = model.predict(frame, conf=0.45, classes=class_IDS, device=0, verbose=False)

# Getting the bounding boxes, confidence, and classes of the recognized objects in the current frame.
boxes = y_hat[0].boxes.xyxy.cpu().numpy()
conf = y_hat[0].boxes.conf.cpu().numpy()
classes = y_hat[0].boxes.cls.cpu().numpy()

# Iterate over the bounding boxes and annotate them on the frame
for box, confidence, class_id in zip(boxes, conf, classes):
    xmin, ymin, xmax, ymax = box.astype(int)
    label = dict_classes[class_id]
    
    # Draw the bounding box
    cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
    
    # Add label and confidence
    text = f"{label} ({confidence:.2f})"
    cv2.putText(frame, text, (xmin, ymin - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

# Display the final annotated image
#cv2.imshow("Annotated Image", frame)

# Count the number of bounding boxes
box_count = len(boxes)

# Display the count on the side of the image
#cv2.putText(frame, f"Number of Workers: {box_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 2)
text = f"Number of Workers: {box_count}"
text_size, _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 3.2, 2)

text_x = 10
text_y = frame.shape[0] - 10

cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 3.2, (0, 0, 255), 3)


# saving the image
cv2.imwrite('ttt.jpg', frame)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.9 torch-1.13.1+cu117 CUDA:0 (Quadro RTX 6000, 24190MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


True